In [1]:
import pyspark
import random
sc = pyspark.SparkContext(appName="Pi")

In [2]:
appliedPatchesPath = "AppliedPatches.txt"
patchesPath = "Patches.txt"
serversPath = "Servers.txt"

appliedPatchesRDD = sc.textFile(appliedPatchesPath)
patchesRDD = sc.textFile(patchesPath)
serversRDD = sc.textFile(serversPath)
output1 = "outPart1/"
output2 = "outPart2/"

In [3]:
ubuntu2 = patchesRDD\
        .filter(lambda elem: elem.split(',')[2] == "Ubuntu2")\
        .map(lambda elem: (elem.split(',')[0], elem.split(',')[1]))

In [4]:
ubuntu2.collect()

[('PIDW15_22', '2020/12/20'),
 ('PIDW10_22', '2022/01/18'),
 ('PIDW11_22', '2022/01/18'),
 ('PID06_21', '2021/01/18'),
 ('PID07_21', '2021/01/18'),
 ('PID08_21', '2021/01/19'),
 ('PID09_21', '2021/01/19'),
 ('PID10_21', '2021/01/19'),
 ('PID11_21', '2021/01/19'),
 ('PID12_21', '2021/01/19'),
 ('PID13_21', '2021/01/19'),
 ('PID14_21', '2021/01/19'),
 ('PID15_21', '2021/01/19'),
 ('PID16_21', '2021/01/19'),
 ('PID17_21', '2021/01/19'),
 ('PID18_21', '2021/01/19')]

In [5]:
serverPatches = appliedPatchesRDD.map(lambda elem: (elem.split(',')[0], elem.split(',')[2]))

In [6]:
serverPatches.collect()

[('PIDW15_22', '2020/12/20'),
 ('PIDW15_22', '2020/12/20'),
 ('PIDW15_22', '2020/12/20'),
 ('PIDW15_22', '2020/12/20'),
 ('PIDW15_22', '2020/12/20'),
 ('PIDW15_22', '2020/12/20'),
 ('PIDW15_22', '2020/12/20'),
 ('PIDW15_22', '2020/12/20'),
 ('PIDW15_22', '2020/12/20'),
 ('PIDW15_22', '2020/12/20'),
 ('PIDW15_22', '2020/12/20'),
 ('PIDW15_22', '2020/12/20'),
 ('PIDW15_22', '2020/12/20'),
 ('PIDW15_22', '2020/12/20'),
 ('PIDW15_22', '2020/12/20'),
 ('PIDW15_22', '2020/12/20'),
 ('PIDW15_22', '2020/12/20'),
 ('PIDW15_22', '2020/12/20'),
 ('PIDW15_22', '2020/12/20'),
 ('PIDW15_22', '2020/12/20'),
 ('PIDW15_22', '2020/12/20'),
 ('PIDW15_22', '2020/12/20'),
 ('PIDW15_22', '2020/12/20'),
 ('PIDW15_22', '2020/12/20'),
 ('PIDW15_22', '2020/12/20'),
 ('PIDW15_22', '2020/12/20'),
 ('PIDW15_22', '2020/12/20'),
 ('PIDW15_22', '2020/12/20'),
 ('PIDW15_22', '2020/12/20'),
 ('PIDW15_22', '2020/12/20'),
 ('PIDW15_22', '2020/12/20'),
 ('PIDW15_22', '2020/12/20'),
 ('PIDW15_22', '2020/12/20'),
 ('PIDW15_

In [9]:
res1 = ubuntu2\
    .join(serverPatches)\
    .filter(lambda elem: elem[1][0] == elem[1][1])\
    .map(lambda elem: (elem[0], 1))\
    .reduceByKey(lambda v1, v2: v1+v2)\
    .filter(lambda elem: elem[1] >= 100)\
    .keys()

In [25]:
res1.collect()

['PIDW15_22', 'PIDW11_22']

In [26]:
# Starting from applied patches rdd, filter only those patches applied in 2021
# and map into a pair RDD with
# key = SID
# value = month
# and perform a distinct operation to keep for each server the distinct months

def filter2021(l):
    fields = l.split(",")
    date = fields[2]
    return date.startswith("2021")

def sidMonth(l):
    fields = l.split(",")
    sid = fields[1]
    date = fields[2]
    month = int(date.split("/")[1])
    return (sid, month)


serverMonthAppliedPatch = appliedPatchesRDD\
            .filter(filter2021)\
            .map(sidMonth)\
            .distinct()

In [27]:
serverMonthAppliedPatch.collect()

[('SID1', 2),
 ('SID1', 4),
 ('SID1', 8),
 ('SID1', 10),
 ('SID2', 1),
 ('SID2', 3),
 ('SID2', 5),
 ('SID2', 7),
 ('SID2', 9),
 ('SID2', 11),
 ('SID1', 1),
 ('SID1', 5),
 ('SID1', 7),
 ('SID1', 9),
 ('SID1', 11),
 ('SID2', 2),
 ('SID2', 4),
 ('SID2', 6),
 ('SID2', 8),
 ('SID2', 10),
 ('SID2', 12)]

In [46]:
# Compute the number of distinct months with at least one applied patch for each server

serverMonths21NumApplPatches = serverMonthAppliedPatch.mapValues(lambda v: 1)\
                                                .reduceByKey(lambda v1, v2: v1+v2)

In [47]:
serverMonths21NumApplPatches.collect()

[('SID1', 9), ('SID2', 12)]

In [48]:
# Calculate the number of distinct months in 2021 without at least one applied patch 
# for each server by applying the formula 12 - number of months with applied patches
serverMonths21NumNoApplPatches = serverMonths21NumApplPatches.mapValues(lambda v: 12-v)

In [50]:
serverMonths21NumNoApplPatches.collect()

[('SID1', 3), ('SID2', 0)]

In [51]:
# serverMonths21NoApplPatches does not contain the servers without applied patches 
# for all the 12 months of 2021

# Use serversRDD to gather information from all servers

# Prepare a pairRDD with
# key = SID
# value = 12
def SID12(l):
    fields = l.split(",")
    sid = fields[0]
    return (sid, 12)



allServers = serversRDD.map(SID12)

In [52]:
allServers.collect()

[('SID1', 12),
 ('SID2', 12),
 ('SID3', 12),
 ('S1', 12),
 ('S2', 12),
 ('S3', 12),
 ('S4', 12),
 ('S5', 12),
 ('S6', 12),
 ('S7', 12),
 ('S8', 12),
 ('S9', 12),
 ('S10', 12),
 ('S11', 12),
 ('S12', 12),
 ('S13', 12),
 ('S14', 12),
 ('S15', 12),
 ('S16', 12),
 ('S17', 12),
 ('S18', 12),
 ('S19', 12),
 ('S20', 12),
 ('S21', 12),
 ('S22', 12),
 ('S23', 12),
 ('S24', 12),
 ('S25', 12),
 ('S26', 12),
 ('S27', 12),
 ('S28', 12),
 ('S29', 12),
 ('S30', 12),
 ('S31', 12),
 ('S32', 12),
 ('S33', 12),
 ('S34', 12),
 ('S35', 12),
 ('S36', 12),
 ('S37', 12),
 ('S38', 12),
 ('S39', 12),
 ('S40', 12),
 ('S41', 12),
 ('S42', 12),
 ('S43', 12),
 ('S44', 12),
 ('S45', 12),
 ('S46', 12),
 ('S47', 12),
 ('S48', 12),
 ('S49', 12),
 ('S50', 12),
 ('S51', 12),
 ('S52', 12),
 ('S53', 12),
 ('S54', 12),
 ('S55', 12),
 ('S56', 12),
 ('S57', 12),
 ('S58', 12),
 ('S59', 12),
 ('S60', 12),
 ('S61', 12),
 ('S62', 12),
 ('S63', 12),
 ('S64', 12),
 ('S65', 12),
 ('S66', 12),
 ('S67', 12),
 ('S68', 12),
 ('S69', 12)

In [53]:
serverNoApplPatches2021 = allServers.subtractByKey(serverMonths21NumApplPatches)

In [55]:
serverNoApplPatches2021.collect()

[('S1', 12),
 ('S2', 12),
 ('S21', 12),
 ('S28', 12),
 ('S37', 12),
 ('S38', 12),
 ('S39', 12),
 ('S42', 12),
 ('S47', 12),
 ('S56', 12),
 ('S57', 12),
 ('S60', 12),
 ('S61', 12),
 ('S71', 12),
 ('S72', 12),
 ('S74', 12),
 ('S78', 12),
 ('S80', 12),
 ('S98', 12),
 ('S4', 12),
 ('S8', 12),
 ('S10', 12),
 ('S13', 12),
 ('S20', 12),
 ('S29', 12),
 ('S33', 12),
 ('S34', 12),
 ('S44', 12),
 ('S53', 12),
 ('S55', 12),
 ('S67', 12),
 ('S69', 12),
 ('S75', 12),
 ('S79', 12),
 ('S84', 12),
 ('S89', 12),
 ('S95', 12),
 ('S102', 12),
 ('S5', 12),
 ('S7', 12),
 ('S16', 12),
 ('S17', 12),
 ('S19', 12),
 ('S23', 12),
 ('S30', 12),
 ('S32', 12),
 ('S35', 12),
 ('S36', 12),
 ('S41', 12),
 ('S43', 12),
 ('S46', 12),
 ('S51', 12),
 ('S54', 12),
 ('S58', 12),
 ('S63', 12),
 ('S65', 12),
 ('S66', 12),
 ('S68', 12),
 ('S73', 12),
 ('S77', 12),
 ('S81', 12),
 ('S85', 12),
 ('S87', 12),
 ('S88', 12),
 ('S93', 12),
 ('S96', 12),
 ('S97', 12),
 ('S100', 12),
 ('SID3', 12),
 ('S3', 12),
 ('S6', 12),
 ('S9', 12)

In [58]:
# The complete result is the union between serverNoApplPatches2021 
# and serverMonths21NumNoApplPatches
res2 = serverNoApplPatches2021.union(serverMonths21NumNoApplPatches)